In [1]:
from sqlalchemy import create_engine, exc
import pandas as pd

In [2]:
usuario = 'root'
senha = 'root'
host = 'localhost'
portorigem = 3306
portdestino = 3306
bancoorigem = 'vendas'
bancodestino = 'vendas_dimensional'

In [3]:
try:
    engineori = create_engine(f"mysql+pymysql://{usuario}:{senha}@{host}:{portorigem}/{bancoorigem}")
    with engineori.connect() as connori:
        print(f"Conexão com o banco '{bancoorigem}' estabelecida com sucesso.")
except exc.SQLAlchemyError as e:
    print(f"Erro ao conectar ao banco '{bancoorigem}': {e}")
    exit()

Conexão com o banco 'vendas' estabelecida com sucesso.


In [4]:
try:
    enginedes = create_engine(f"mysql+pymysql://{usuario}:{senha}@{host}:{portdestino}/{bancodestino}")
    with enginedes.connect() as conndes:
        print(f"Conexão com o banco '{bancodestino}' estabelecida com sucesso.")
except exc.SQLAlchemyError as e:
    print(f"Erro ao conectar ao banco '{bancodestino}': {e}")
    exit()

Conexão com o banco 'vendas_dimensional' estabelecida com sucesso.


In [5]:
sql = f"""
    select ve.id,ve.nome,ve.sexo,ve.nascimento,ve.datacontrato,vu.nome as supervisor
    from vendedores ve
    left join vendedores vu on ve.supervisor = vu.id
    """
vendedores = pd.read_sql(sql,engineori)
vendedores = vendedores.reset_index()
vendedores.columns=['id','idtransacional','nome','sexocodigo','nascimento','contrato','supervisor']
vendedores['id'] += 1
vendedores['sexodescricao'] = vendedores['sexocodigo'].map({
    'f': 'Feminino',
    'F': 'Feminino',
    'm': 'Masculino',
    'M': 'Masculino',
    'o': 'Outros',
    'O': 'Outros'
}).fillna('Não Informado')
vendedores['supervisor'].fillna('Sem supervisor')
try:
    vendedores.to_sql('dim_vendedor',if_exists='append',index=False,con=enginedes)
    print("Dimensão vendedor carregada com sucesso.")
except exc.SQLAlchemyError as e:
        print(f"Erro ao carregar: {e}")

Dimensão vendedor carregada com sucesso.
